# Model run | "The role of sufficiency measures in a decarbonizing Europe"

This notebook guides you through the process of running all the scenarios for the study

> **"The role of sufficiency measures in a decarbonizing Europe"**  
> Published: 29 April 2025, Ecological Economics  
> DOI: [10.1016/j.ecolecon.2025.108645](https://doi.org/10.1016/j.ecolecon.2025.108645)

---

## About the Zenodo Repository

The Zenodo repository contains all data and results related to the study.  
**[Zenodo Link](https://zenodo.org/records/15261005)**


---

## 1. Parse the MARIO Database

This cell parses the MARIO database. Please run the first two notebooks before this step. Note that you can directly download the database from the Zenodo repository as shown in notebook 2.
It uses the `mario` package to load the Supply-Use Table (SUT) in "flows" mode. 

In [ ]:
import mario
import pandas as pd
import os
import json
import time
import warnings 
warnings.filterwarnings("ignore") # Pandas became a bit noisy with warnings

db = mario.parse_from_txt(
    path = "Data/MARIO database/flows", # it can be also directly downloaded from Zenodo, skipping step 1. Database building
    table = 'SUT', 
    mode ='flows',
)

### Preparing Shock Calculation

The cell below defines the necessary variables and structures for preparing the shock calculation. Key components include:

- **Involved Commodities (`inv_commodities`)**: A list of 15 specific commodities involved in the analysis, such as meat products, dairy, and electricity.
- **Involved Regions (`inv_regions`)**: A list of 27 European regions included in the study.
- **Clusters (`clusters`)**: A dictionary organizing data into categories like `Commodity`, `Activity`, `Region`, and `Consumption category`. Each category contains all available elements and subsets of involved elements.
- **Column Definitions**: Lists like `U_cols`, `S_cols`, `Y_cols`, etc., define the structure of various matrices used in the analysis.
- **Result Mapping (`Res_map`)**: A dictionary mapping matrix names (e.g., `V`, `F`, `E`) to their column definitions and export names.

This setup ensures that the shock calculation process is well-structured and ready for execution in subsequent cells.

In [ ]:
#%% Preparing shock calculation
inv_commodities = ['Products of meat cattle','Products of meat pigs','Products of meat poultry',
                   'Dairy products','Fish products','Vegetables; fruit; nuts','Food products nec',
                   'products of Vegetable oils and fats','Beverages','Sugar','Liquid fuels',
                   'Liquefied Petroleum Gases (LPG)','Natural gas and services related to natural gas extraction; excluding surveying',
                   'Chemicals nec','Electricity']

inv_regions = ['AT', 'BE', 'BG', 'CY', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR', 'GR', 'HR', 'HU', 'IE', 'LT', 'LU', 'LV', 'MT', 'NL', 'PL', 'PT', 'RO', 'SE', 'SI', 'SK', 'IT']

# Defining the clusters
clusters = {
'Commodity':{
    'all':db.get_index('Commodity'),
    'Involved commodities': inv_commodities,
    },
'Activity':{
    'all':db.get_index('Activity'),
    },
'Region':{
    'all':db.get_index('Region'),
    'Involved regions': inv_regions,
    },
'Consumption category':{'all':db.get_index('Consumption category')},
}

# Structuring the export of the results
U_cols = ['Region_from','drop','Commodity','Activity', 'drop','Region_to']
S_cols = ['Region_from','drop','Activity','Commodity', 'drop','Region_to']
Y_cols = ['Region_from','drop','Commodity','Consumption category', 'drop','Region_to']
EY_cols = ['Satellite account','Consumption category','drop','Region_to']
E_cols = ['Satellite account','Activity','drop','Region_to']
F_cols = ['Satellite account','Commodity','drop','Region_to']
V_cols = ['Factor of production','Activity','drop','Region_to']
X_cols = ['Region_from','drop','Activity']
Q_cols = ['Region_from','drop','Commodity']

Res_map = {
        'V': {'col_names':V_cols, 'exp_name': 'VA'},
        'F': {'col_names':F_cols, 'exp_name': 'R_e'},
        'e': {'col_names':E_cols, 'exp_name': 'rr'},
        'E': {'col_names':E_cols, 'exp_name': 'R'},
        'EY': {'col_names':EY_cols, 'exp_name': 'R_hh'},
        'f': {'col_names':E_cols, 'exp_name': 'r_ee'},
        'Q': {'col_names':Q_cols, 'exp_name': 'Q'},
        'X': {'col_names':X_cols, 'exp_name': 'X'}
        }

## 3. Shock Calculation and Results Extraction

This section runs the shock calculations and exports the results:

- **Scenario Definition**: Scenarios are defined by combinations of background, measure, and year.
- **Calculation and Export Options**:  
  - `calc`: If `True`, shocks are calculated.  
  - `export`: If `True`, results are exported.  
  - `forget`: If `True`, each scenario is deleted from memory after export to avoid memory issues.
- **Result Aggregation**: Results are aggregated for visualization and saved in the `Outputs/Results` directory.


In [ ]:
#%% Shock calculation and results extraction

# Defining the matrices to be exported
ex_mat = ['V','F','e','E','EY','f','Q','X']

# Define which scenarios to calculate. Scenarios are defined by the combination of background, measure and year.
background = [
    'REF',  # Reference scenario
    'NZE',  # Quicker transition
    'BAU'   # Business as usual
    ] 
measure = [
    'All',  # All measures
    'D',    # Diets
    'S',    # Sharing spaces
    'M',    # Car efficiency
    'P',    # Sharing products
    'F',    # Flying less
    'B',    # Cycling more
    0,      # No measure
    ] 
years = [
    '2011',
    '2020',
    '2025',
    '2030',
    '2035',
    '2040',
    '2045',
    '2050',
    ] # 5 year steps from 2020 to 2050 (pass list of strings)


calc = True  # If True,the shocks are calculated. If False, it assumes the calculation are already done and that only export is needed.
export = True # If True, the results are exported. If False, it assumes the export is not needed.
forget = True # If True, each scenario is deleted after being calculated and exported. This option avoid memory issues in case many scenarios (i.e. more than 10) are calculated. If False, the scenarios are kept in memory

res_agg = pd.read_excel("Outputs/sets.xlsx", sheet_name=None) # Da finire per aggregare

for y in years:
    for b in background:
        for m in measure:
            
            start = time.time()

            if calc:
                if forget:
                    scemario = 'SCEMARIO'
                else:
                    scemario = f'{b}_{m}_{y}'

                db.shock_calc(
                f"Shocks/filled_files/{b}_{m}_{y}.xlsx",
                z=True,
                Y=True,
                e=True,
                scenario=scemario,
                force_rewrite=True,
                **clusters,
                )

                end = time.time()
                print(f"Scemario {b}_{m}_{y} calculated in {round(end-start, 2)} seconds.")


            if export:
                for mat in ex_mat:
                    if mat==ex_mat[-1]:    
                        print(mat)              
                    else:
                        print(mat,end=" ")              

                    if not os.path.exists(f"Outputs/Results/{Res_map[mat]['exp_name']}"):
                        os.makedirs(f"Outputs/Results/{Res_map[mat]['exp_name']}")
                    if mat not in ['X','Q']:
                        data = db.query(matrices=[mat], scenarios=[scemario]).stack().stack().stack()
                    else:
                        if mat == 'X':
                            data = db.query(matrices=['X'], scenarios=[scemario]).loc[(slice(None),"Activity",slice(None)),:]
                        if mat == 'Q':
                            data = db.query(matrices=['X'], scenarios=[scemario]).loc[(slice(None),"Commodity",slice(None)),:]

                    data.index.names = Res_map[mat]['col_names']
                    drop_levels = [i for i, name in enumerate(data.index.names) if name == 'drop']
                    data = data.reset_index(level=drop_levels, drop=True)
                    if mat not in ['X','Q']:
                        data = data.to_frame().reset_index()
                        data = data.rename(columns={0: 'Value'})
                    else:
                        data = data.rename(columns={'production': 'Value'})
                                    
                    # Aggregating the results for visualization
                    if 'Activity' in data.columns:
                        cols_dict = dict(zip(res_agg['_set_ACTIVITIES']['Activity name'], res_agg['_set_ACTIVITIES']['Activity_PBI']))
                        data['Activity'] = data['Activity'].map(cols_dict)
                        data.set_index('Activity', inplace=True, append=True)

                    if 'Commodity' in data.columns:
                        cols_dict = dict(zip(res_agg['_set_COMMODITIES']['Commodity name'], res_agg['_set_COMMODITIES']['Commodity_PBI']))
                        data['Commodity'] = data['Commodity'].map(cols_dict)
                        data.set_index('Commodity', inplace=True, append=True)
                    
                    data.set_index([col for col in data.columns if col != 'Value'], inplace=True, append=True)
                    data = data.groupby(level=list(range(data.index.nlevels))).sum().reset_index()

                    data['Year'] = y
                    data['Scenario'] = f'{b}_{m}'
                    if mat not in ['X','Q']:
                        data = data.drop('level_0', axis=1)

                    mat_file = f"Outputs/Results/{Res_map[mat]['exp_name']}/{b}_{m}_{y}.txt"
                    data.to_csv(mat_file, index=False, sep='\t')

                    end = time.time()
                    print(f"Scemario {b}_{m}_{y}: {mat} exported in {round(end-start, 2)} seconds.")


print('Finished!')


## 4. Merge Results (Optional)

This section merges all exported results for each matrix into a single file for easier analysis and visualization.  
Merged files are saved in `Outputs/Results/Merged`.

In [ ]:
# Checking if the folder exists
results_folder = f"Outputs/Results/Merged"
if not os.path.exists(results_folder):
    os.makedirs(results_folder)

for mat, names in Res_map.items():

    if mat not in ['U','S']:

        files = os.listdir(f"Outputs/Results/{names['exp_name']}")
        # Placeholder code
        dataframes = []
        for file in files:
            if file.endswith('.txt'):
                file_path = f"{paths['Results']}/{Out_Db}/{names['exp_name']}/{file}"
                df = pd.read_csv(file_path, sep='\t')
                dataframes.append(df)

        merged_df = pd.concat(dataframes, axis=0)
        merged_df.to_csv(f"{results_folder}/{names['exp_name']}.txt", sep='\t', index=False)
        print(f'{mat} exported ')